In [2]:
import numpy as np
import S4

In [ ]:
def blazedGratingRnS4(lda,angle,phi,pola,
        a,theta1,theta2,numLayer,nResist,kResist,dResist,nGlass,kGlass,numBasis):
    '''
    Define a function of S4 for calculating the diffraction efficiency of AR.
    args: 
        for the light: lda, wavelength; angle, incident angle; phi, azimuth angle; pola, polarization;
        for the structure: a, period; theta1, blazed angle; theta2, another angle; numLayer, number of rectangle layers forming the grating;
        for the resist layer: nResist, real part of refractive index; kResist, imaginary part of refractive index; dResist, thickness;
        for the substrate: nSubs, real part of refractive index; kSubs, imaginary part of refractive index;
        for the Fourier: numBasis, number of Fourier expansion series
    returns:
        R: diffraction efficiency of specific order;
    '''
    period = [a,0]
    S = S4.New(Lattice=((period[0],0), (0,period[1])), NumBasis=numBasis)
    S.SetMaterial(Name='Air', Epsilon=(1+0j)**2)
    S.SetMaterial(Name='Resist', Epsilon=(nResist+1j*kResist)**2)
    S.SetMaterial(Name='Glass', Epsilon=(nGlass+1j*kGlass)**2)

    S.AddLayer(Name='Air_layer', Thickness=1, Material='Air')
    h = a / (1/np.tan(theta1*np.pi/180)+1/np.tan(theta2*np.pi/180))
    dw = a / numLayer
    dh = h / numLayer
    for l in range(1, numLayer+1):
        S.AddLayer(Name='Gratings'+str(l)+'_layer', Thickness=dh, Material='Air')
        S.SetRegionRectangle(
            Layer='Gratings'+str(l)+'_layer',
            Material='Resist',
            Center=((h/np.tan(theta1*np.pi/180)-a/2)*(numLayer-l)/numLayer, 0),
            Angle=0,
            Halfwidths=(dw*l/2, 0)
        )

    S.AddLayer(Name='Resist_layer', Thickness=dResist-h, Material='Resist')
    S.AddLayer(Name='Glass_layer', Thickness=1, Material='Glass')

    S.SetExcitationPlanewave(
    IncidenceAngles=(angle,phi), sAmplitude=pola, pAmplitude=1-pola, Order=0)
    S.SetOptions(PolarizationDecomposition=True)
    S.SetFrequency(1/lda)
    powr = S.GetPowerFluxByOrder(Layer='Air_layer', zOffset=0)

    R0 = np.abs(np.real(powr[0][1])) / np.real(powr[0][0])
    Rn1 = np.abs(np.real(powr[2][1])) / np.real(powr[0][0])
    Rn2 = np.abs(np.real(powr[4][1])) / np.real(powr[0][0])
    Rn3 = np.abs(np.real(powr[6][1])) / np.real(powr[0][0])

    return R0, Rn1, Rn2, Rn3